# Data Science - Segmentez des clients d'un site e-commerce

# Auteur : Mme Maty KANE

# Date : le 18 Mars 2025

# __________________________________________________________________________________________

**Liste de requêtes SQL pour le dashboard :**

● En excluant les commandes annulées, quelles sont les commandes
récentes de moins de 3 mois que les clients ont reçues avec au moins 3
jours de retard ?

● Qui sont les vendeurs ayant généré un chiffre d'affaires de plus de 100
000 Real sur des commandes livrées via Olist ?

● Qui sont les nouveaux vendeurs (moins de 3 mois d'ancienneté) qui
sont déjà très engagés avec la plateforme (ayant déjà vendu plus de 30
produits) ?

● Question : Quels sont les 5 codes postaux, enregistrant plus de 30
reviews, avec le pire review score moyen sur les 12 derniers mois ?

In [6]:
pip install ipython-sql sqlite3 pandas

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3


# __________________________________________________________________________________________

In [8]:
import sqlite3
import pandas as pd

# Définnition du chemin de la base de données
db_path = r"C:\Users\matym\Desktop\FORMATION\PROJET 5\Projet à faire\olist.db"

# Connexion à la base SQLite
conn = sqlite3.connect(db_path)

# __________________________________________________________________________________________

**Requête pour :**
* **filtrer les commandes non annulées,**
* **et en retard d'au moins 3 jours par rapport à la date de livraison estimée,**
* **tout en considérant uniquement les commandes des 3 derniers mois.**

In [11]:
query = """
WITH last_order_date AS (
    SELECT MAX(order_purchase_timestamp) AS max_date FROM orders
)
SELECT * FROM orders
WHERE order_status != 'canceled'
AND STRFTIME('%Y-%m-%d', order_delivered_customer_date) >= STRFTIME('%Y-%m-%d', order_estimated_delivery_date, '+3 days')
AND STRFTIME('%Y-%m-%d', order_purchase_timestamp) >= 
    (SELECT DATE(max_date, '-3 months') FROM last_order_date);
"""

# Exécuter la requête et stocker le résultat dans un DataFrame
df = pd.read_sql_query(query, conn)

# Affichage des 5 premières lignes
print(df.head())

   index                          order_id                       customer_id  \
0    561  cfa4fa27b417971e86d8127cb688712f  7093250e1741ebbed41f0cc552025fd6   
1    711  234c056c50619f48da64f731c48242b4  44e460a655f7154ccd9faa4dbbbaf68a   
2    728  8ad3f1d0f96992e43566c4c82c9f6c58  948b29e24216a05fea13a18d8db45ea5   
3   1844  7f579e203c931f3e8410103359c6d523  d665be250d1c687c58fdea61a9b55a58   
4   1967  cb6e441ff2ef574ce08d3709426f88ec  4fb843d304c57182d4aa27bb39ca592b   

  order_status order_purchase_timestamp    order_approved_at  \
0    delivered      2018-08-16 09:44:23  2018-08-17 03:10:22   
1    delivered      2018-08-14 14:49:15  2018-08-15 03:15:15   
2    delivered      2018-07-17 21:25:29  2018-07-17 21:35:17   
3    delivered      2018-08-02 18:09:27  2018-08-02 18:43:43   
4    delivered      2018-08-08 19:27:03  2018-08-08 19:44:59   

  order_delivered_carrier_date order_delivered_customer_date  \
0          2018-08-17 18:55:00           2018-08-29 01:41:41   
1     

# __________________________________________________________________________________________

* **Requête pour :**
* **obtenir les vendeurs ayant un chiffre d’affaires supérieur à 100 000,** 
* **en filtrant uniquement les commandes livrées.**

In [14]:
query = """
SELECT s.seller_id, s.seller_city, s.seller_state, 
       SUM(oi.price) AS chiffre_affaires 
FROM order_items AS oi 
JOIN orders o ON oi.order_id = o.order_id 
JOIN sellers s ON oi.seller_id = s.seller_id 
WHERE o.order_status = 'delivered' 
GROUP BY s.seller_id, s.seller_city, s.seller_state 
HAVING chiffre_affaires > 100000 
ORDER BY chiffre_affaires DESC;
"""

# Exécuter la requête et stocker le résultat dans un DataFrame
df = pd.read_sql_query(query, conn)

# Affichage des 5 premières lignes
print(df.head())

                          seller_id       seller_city seller_state  \
0  4869f7a5dfa277a7dca6462dcf3b52b2           guariba           SP   
1  53243585a1d6dc2643021fd1853d8905  lauro de freitas           BA   
2  4a3ca9315b744ce9f8e9374361493884          ibitinga           SP   
3  fa1c13f2614d7b5c4749cbc52fecda94            sumare           SP   
4  7c67e1448b00f6e969d365cea6b010ab   itaquaquecetuba           SP   

   chiffre_affaires  
0         226987.93  
1         217940.44  
2         196882.12  
3         190917.14  
4         186570.05  


# __________________________________________________________________________________________

**Requête pour :**
* **récupèrer les vendeurs ayant effectué leur première vente au cours des 3 derniers mois (par rapport au 29 août 2018),**
* **et ayant vendu plus de 30 produits.**

In [17]:
query = """
WITH first_sale AS (
    SELECT oi.seller_id, 
           MIN(o.order_purchase_timestamp) AS first_order_date 
    FROM order_items oi 
    JOIN orders o ON oi.order_id = o.order_id 
    GROUP BY oi.seller_id
), 
seller_activity AS ( 
    SELECT oi.seller_id, COUNT(*) AS total_products_sold 
    FROM order_items oi
    GROUP BY oi.seller_id
) 
SELECT f.seller_id, s.seller_city, s.seller_state, 
       f.first_order_date, sa.total_products_sold 
FROM first_sale f 
JOIN seller_activity sa ON f.seller_id = sa.seller_id 
JOIN sellers s ON f.seller_id = s.seller_id 
WHERE f.first_order_date >= DATE('2018-08-29', '-3 MONTH') 
AND sa.total_products_sold > 30;
"""

# Exécuter la requête et stocker le résultat dans un DataFrame
df = pd.read_sql_query(query, conn)

# Affichage des 5 premières lignes
print(df.head())

                          seller_id      seller_city seller_state  \
0  18e694e0e48ed6f7aa3f24aade5fd697  varzea paulista           SP   
1  240b9776d844d37535668549a396af32          niteroi           RJ   
2  31561f325664a8a7aba4c8d0c3a9b3db        sao paulo           SP   
3  3ab971ce71839580d2ae5b4e40fe8044        sao paulo           SP   
4  6061155addc1e54b4cfb51c1c2a32ad8         campinas           SP   

      first_order_date  total_products_sold  
0  2018-06-05 19:28:28                   34  
1  2018-07-17 13:48:59                   36  
2  2018-06-05 22:51:12                   38  
3  2018-06-03 17:41:43                   38  
4  2018-06-01 01:20:55                   33  


# __________________________________________________________________________________________

**Requête pour :**
* **identifier les 5 zones postales (ZIP) avec les pires moyennes de satisfaction client,** 
* **sur les 12 derniers mois,**
* **en ne considérant que les codes postaux ayant reçu plus de 30 avis.**

In [19]:
query = """
WITH last_review_date AS ( 
    SELECT MAX(review_creation_date) AS max_date 
    FROM order_reviews
), 
filtered_reviews AS ( 
    SELECT c.customer_zip_code_prefix,
           AVG(orv.review_score) AS avg_review_score, 
           COUNT(orv.review_id) AS total_reviews 
    FROM order_reviews orv 
    JOIN orders o ON orv.order_id = o.order_id 
    JOIN customers c ON o.customer_id = c.customer_id
    WHERE orv.review_creation_date >= (SELECT DATE(max_date, '-12 months') FROM last_review_date) 
    GROUP BY c.customer_zip_code_prefix 
    HAVING total_reviews > 30
) 
SELECT customer_zip_code_prefix, avg_review_score, total_reviews 
FROM filtered_reviews 
ORDER BY avg_review_score ASC 
LIMIT 5;
"""

# Exécuter la requête et stocker le résultat dans un DataFrame
df = pd.read_sql_query(query, conn)

# Affichage des résultats
print(df)

# Fermer la connexion après exécution
conn.close()

   customer_zip_code_prefix  avg_review_score  total_reviews
0                     22753          2.867925             53
1                     22723          3.000000             31
2                     28893          3.125000             32
3                     22770          3.184211             38
4                     13056          3.272727             33
